In [ ]:
import nltk
import numpy as np
import enchant
import pandas as pd
import os
import re
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')      
nltk.download('averaged_perceptron_tagger')

In [ ]:
data = []
# ścieżka do plików
p = os.path.join(os.getcwd(), 'data')
for folder in os.listdir(p):
    path = os.path.join(p, folder)
    # jeśli plik to pomiń
    if os.path.isfile(path):
        continue
    # jesli folder to pobierz pliki
    mails = os.listdir(path)
    for mail in mails:
        path = os.path.join(p, folder, mail)
        with open(path,"r") as file:
            text = file.read()
        data.append([text, folder])
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
def Preprocesing(data, lematize = False, stem = True):
    
    # Bierzemy wszystko po pierwszych 2 enterach czyli po opisie technicznym
    formated = data.split('\n\n')[1:]
    formated = ' '.join(formated).lower() # zamiana na małe litery
    
    # Wyrzucamy wszystkie wyrażenia słowo.słowo(ścieżki itp.) i adresy mail 
    formated = re.sub('((\w+\.)+\w+)|\w+@\w+','', formated)
    
    # Podmieniamy wszystkie liczby, ciągi liczb na spacje  i "_"
    formated =re.sub('\W|\d|_'," ", formated)
    
    # Tokenizacja
    tokenizer = RegexpTokenizer(r"\w+")
    formated= tokenizer.tokenize(formated)
    
    # Usuwamy stop wordsy
    stop_words = set(stopwords.words('english'))
    formated = [word for word in formated if not word in stop_words and len(word) > 1]
    
    # Lematyzacja
    if(lematize == True):
        lemmatizer = WordNetLemmatizer()
        formated = [lemmatizer.lemmatize(word) for word in formated] 
    
    # Stemming    
    if (stem == True):
        ps = PorterStemmer() 
        formated = [ps.stem(word) for word in formated] 
        
    return formated

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.DataFrame(data)

X_train, X_test, y_train, y_test = train_test_split(
    df[0],
    df[1],
    test_size=0.4, random_state=213)

# Preprocesing
X_train = [Preprocesing(text) for text in X_train]
X_train

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# max i min trzeba zhiperparametryzować wyrzucilismy slowa ktore istnieja w mniej niz 1 procencie tekstow i wiecej niz 0.2

tf_idf = [" ".join(text) for text in X_train]
vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english', max_df = 0.2 ,min_df=0.01)
tf_idf = vectorizer.fit_transform(tf_idf)
tf_idf = pd.DataFrame(tf_idf.toarray(), columns = vectorizer.get_feature_names())
tf_idf

In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN().fit(X)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(14,8)).clf()
sns.histplot(clustering.labels_)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters =  5, random_state = 0)
kmeans.fit(X)

In [ ]:
plt.figure(figsize=(14,8)).clf()
sns.histplot(kmeans.labels_)

In [ ]:
## Tu jeszcze kombinuje bo wcześniej głupot narobiłem

In [ ]:
# Pewnie do śmieci
def modeling(df, number_of_topics):
    from sklearn.feature_extraction.text import CountVectorizer
    
    #matrix = CountVectorizer()
    #X = matrix.fit_transform(df).toarray()
    #b_of_words = pd.DataFrame(X, columns = matrix.get_feature_names_out())
    #tfidf = TfidfVectorizer(lowercase=True,
     #                   stop_words='english',
    #                    ngram_range = (1,1))

    # Fit and Transform the documents
    train_data = tfidf.fit_transform(df)  
    lsa = TruncatedSVD(n_components=number_of_topics, random_state=42)

    # Fit SVD model on data
    lsa.fit_transform(train_data)
    #b_of_words = df.columns
    #terms = tfidf.get_feature_names_out()

    for index, component in enumerate(lsa.components_):
        zipped = zip(terms, component)
        top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:10]
        top_terms_list=list(dict(top_terms_key).keys())
        print("Topic "+str(index)+": ",top_terms_list)
        
    m1= np.array(lsa.components_.T)
    m2= np.array(b_of_words)
    X = np.dot(m2,m1)
    X = pd.DataFrame(X)
    return X

In [ ]:
X = modeling(tf_idf, 10)

In [ ]:
tf_idf